In [1]:
import os
from dotenv import load_dotenv

from openai import AsyncAzureOpenAI
from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread, AzureAIAgent, AzureAIAgentThread, SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.core_plugins import ConversationSummaryPlugin, MathPlugin

# Setup
load_dotenv(override=True)
azure_chat_completion = AzureChatCompletion(
    service_id="azure_openai_chat",
    async_client=AsyncAzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("MODEL_DEPLOYMENT_API_VERSION"),
    ),
    deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME"),
)

# Part 1: Create a single agent, a thread and run the agent

## ChatCompletionAgent

In [2]:
# 1. Create the agent
teacher_agent = ChatCompletionAgent(
    service=azure_chat_completion,
    name="TeacherAssistant",
    instructions="Answer the math questions of the student.",
    plugins=[MathPlugin()],
)

# 2. Create the thread
thread: ChatHistoryAgentThread = ChatHistoryAgentThread()

# 3. Run the agent as a chat
while True:
    user_input = input("Enter your message:")
    print(f"\nYou: {user_input}")

    if user_input.lower() in ["exit", "quit"]:
        break

    response = await teacher_agent.get_response(
        messages=user_input,
        thread=thread, 
    )

    assistant_reply = str(response)
    print(f"Bot: {assistant_reply}")


You: hi
Bot: Hello! How can I assist you with your math questions today?

You: how are ypu?
Bot: I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?

You: what is 1+1
Bot: The sum of 1 + 1 is 2.0.

You: 
Bot: Do you have any more math questions or anything else you'd like to know?

You: quit


### 🧪 Try it out!

Now that you’ve created a `ChatCompletionAgent`, try chatting with it! 💬

You’ll notice it’s much simpler than using the kernel directly — the agent takes care of prompts, memory, and tool orchestration for you. 🧠⚙️

> 💡 **Did you know?** If you assign a kernel to the agent, it can automatically use everything configured in that kernel — including plugins, services, filters, and memory.



## AzureAIAgent​

In [3]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds, endpoint=os.getenv("AIPROJECT_ENDPOINT")) as client,
):
    # 1. Define an agent on the Azure AI agent service 
    agent_definition = await client.agents.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="SecretaryAssistant",
        instructions="You are a summarization assistant. Help users understand long content by summarizing it.",
    )

    # 2. Create an agent instance with the definition and plugins
    secretary_agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[ConversationSummaryPlugin]
    )

    # 3. Create a thread on the Azure AI agent service 
    thread: AzureAIAgentThread = AzureAIAgentThread(client=client)

    # 4. Run the agent as a chat
    while True:
        user_input = input("Enter your message:")
        print(f"\nYou: {user_input}")

        if user_input.lower() in ["exit", "quit"]:
            break

        response = await secretary_agent.get_response(
            messages=[user_input], thread=thread
        )

        assistant_reply = str(response)
        print(f"Bot: {assistant_reply}")


You: Hello
Bot: Hi there! How can I assist you today?

You: What are you doing?
Bot: I'm here to assist you with any questions or tasks you might have. How can I help you?

You: Can you summarize everything?
Bot: Sure! Could you please provide the text or content you would like summarized?

You: this conversation
Bot: It seems there was an error in processing the request. Let me summarize the conversation for you:

1. You greeted by saying "Hello."
2. I responded, offering assistance.
3. You asked what I was doing.
4. I explained my purpose is to assist with questions or tasks.
5. You requested a summary of our conversation.
6. I asked for the specific content you'd like summarized.

You: exit


### 🧪 Try it out!

Now try interacting with your `AzureAIAgent`! 💬

💡 **Did you know?** When you create the agent and thread, Semantic Kernel registers them in your Azure AI Foundry project. The Azure AI Agent Service automatically persists both the agent definition and conversation thread—no manual storage required.

---

### 🧩 Challenge

1. Enhance your `AzureAIAgent` by adding advanced built-in tools such as:
   - BingGrounding
   - File Search
   - Code Interpreter
   - OpenAPI-based integrations  
   (all supported by Azure AI Agent Service)

2. Explore other **agent types** available in Semantic Kernel’s Python SDK, including:
   - `ChatCompletionAgent`
   - `OpenAIAssistantAgent`
   - `AzureAIAgent`
   - `OpenAIResponsesAgent`  
   These agents vary by their underlying LLM connector and orchestration capabilities.


# Part 2: Multi-agent orchestration

- Setup

In [4]:
def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"# {message.name}\n\n{message.content}\n")
    
def human_response_function() -> ChatMessageContent:
    user_input = input("User: ")
    return ChatMessageContent(role=AuthorRole.USER, content=user_input)

- Define the agents

In [5]:
sequential_agents = [
    ChatCompletionAgent(
        service=azure_chat_completion,
        name="ConceptExtractorAgent",
        instructions=(
            "You are a marketing analyst. Given a product description, identify:\n"
            "- Key features\n"
            "- Target audience\n"
            "- Unique selling points\n\n"
        ),
    ),
    ChatCompletionAgent(
        service=azure_chat_completion,
        name="WriterAgent",
        instructions=(
            "You are a marketing copywriter. Given a block of text describing features, audience, and USPs, "
            "compose a compelling marketing copy (like a newsletter section) that highlights these points. "
            "Output should be short (around 150 words), output just the copy as a single text block."
        ),        
    ),
    ChatCompletionAgent(
        service=azure_chat_completion,
        name="FormatProofAgent",
        instructions=(
            "You are an editor. Given the draft copy, correct grammar, improve clarity, ensure consistent tone, "
            "give format and make it polished. Output the final improved copy as a single text block."
        )
    )
]

- Run the orchestration 

In [6]:
# 1. Create the orchestration
sequential_orchestration = SequentialOrchestration(
    members=sequential_agents,
    agent_response_callback=agent_response_callback,
)

# 2. Create and run the orchestration runtime
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task
orchestration_result = await sequential_orchestration.invoke(
    task="An eco-friendly stainless steel water bottle that keeps drinks cold for 24 hours",
    runtime=runtime,
)

# 4. Get the final result
value = await orchestration_result.get(timeout=50)
print(f"***** Final Result *****\n{value}")

# ConceptExtractorAgent

Key Features:
- Made from stainless steel
- Eco-friendly materials
- Insulation that maintains cold temperatures for up to 24 hours

Target Audience:
- Environmentally conscious consumers 
- Individuals looking for durable, high-quality drinkware
- Outdoor enthusiasts, athletes, and travelers seeking temperature retention in water bottles

Unique Selling Points:
- Long-lasting temperature retention (24 hours for cold drinks)
- Sustainable, eco-friendly materials
- Durable stainless steel construction

# WriterAgent

Stay refreshed and sustainable with our state-of-the-art EcoFlow Flask, crafted for those who demand durability and temperature retention without compromising on eco-friendliness. Made from premium stainless steel, our flask is designed to cater to the needs of outdoor enthusiasts, athletes, and travelers who seek adventure with every sip. Trust in our advanced insulation technology to keep your drinks crisp and cold for up to 24 hours, ensuring you

### 🧪 Try it out!

Now that you've created a **sequential orchestration**, go ahead and run it! 🔄  
You’ll observe each agent executing in turn, passing its output to the next—perfect for linear workflows.


### 🧩 Challenge

- Try other orchestration patterns available in SK:
  - **Concurrent** – broadcast a task to all agents and collect responses in parallel.
  - **Handoff** – dynamically transfer control based on context or conditions.
  - **Group Chat** – multiple agents participate in a shared conversation under coordination.
- Add **human-in-the-loop** support: insert `human_response_function` to pause and await user feedback between steps. :contentReference[oaicite:1]{index=1}  
- **Food for thought:** Could you mix sequential, handoff, and human-in-the-loop patterns to model complex real‑world processes?

All orchestration types share a unified interface—so switching patterns is seamless and requires minimal code changes.
